In [335]:
import pandas as pd
import requests
import json
import numpy as np

def searching(
    query_string,
    broad='no',
    has_ngrams='true',
    does_date_matter = 'yes',
    from_date='2010-01-01',
    to_date='2022-12-12'):

    if broad == 'no':
        if does_date_matter == 'yes':
            institution = requests.get(
                f'https://api.openalex.org/works?filter=abstract.search:{query_string},has_ngrams:{has_ngrams},from_publication_date:{from_date},to_publication_date:{to_date}&page=1&per-page=20'
            ).json()
        else:
            institution = requests.get(
                f'https://api.openalex.org/works?filter=abstract.search:{query_string},has_ngrams:{has_ngrams}&page=1&per-page=20'
            ).json()

    elif broad == 'yes':
        institution = requests.get(
            f'https://api.openalex.org/works?search={query_string}&filter=has_ngrams:{has_ngrams}&page=1&per-page=20'
        ).json()
    
    a = []
    for x in range(20):
        y = institution['results'][x]['ngrams_url']
        if x < 3:
            print(institution['results'][x]['title'])
            print(institution['results'][x]['authorships'][0]['author']['display_name'])
            print(institution['results'][x]['doi'])
            print(institution['results'][x]['relevance_score'])
            print(institution['results'][x]['publication_date'])
            print(y,'\n')  
        z = requests.get(y).json()
        a.append(z)


    return a

_ngrams = searching('environmental flows',broad='no')

# for idx,i in enumerate(_ngrams):
#     if idx == 0:
#         j = i['ngrams']
#         k = pd.DataFrame.from_dict(j)
#     else:
#         j = i['ngrams']
#         l = pd.DataFrame.from_dict(j)
#         k = pd.concat([k,l])

Four billion people facing severe water scarcity
Mesfin Mekonnen
https://doi.org/10.1126/sciadv.1500323
1654.4683
2016-02-01
https://api.openalex.org/works/W2260274902/ngrams 

The ecological limits of hydrologic alteration (ELOHA): a new framework for developing regional environmental flow standards
N. LeRoy Poff
https://doi.org/10.1111/j.1365-2427.2009.02204.x
1344.5311
2010-01-01
https://api.openalex.org/works/W2156353061/ngrams 

A comparative literature analysis of definitions for green and sustainable supply chain management
Payman Ahi
https://doi.org/10.1016/j.jclepro.2013.02.018
727.77094
2013-08-01
https://api.openalex.org/works/W1990457952/ngrams 



In [336]:
ngram = []

for i in _ngrams:
    for j in i['ngrams']:
        ngram.append((j['ngram'].lower(),j['ngram_count']))

print(len(ngram))

b = Counter()
for ng,co in ngram:
    b[ng] += co

c = [[x,y] for x,y in b.items()]

badwords = ['add words to filter as necessary',' .$', 'et al','et$']

df = pd.DataFrame.from_records(c,columns=['ngram','ngram_count'])
df['ngram_tokens'] = df['ngram'].str.split().str.len() #https://stackoverflow.com/questions/37483470/how-to-calculate-number-of-words-in-a-string-in-dataframe
df1 = df.dropna(axis=0)
df2 = df1.loc[(df1['ngram_tokens'] > 1) & (df1['ngram_tokens'] < 5)]
df3 = df2.sort_values(by=['ngram_count','ngram_tokens'],ascending=False)

for i in badwords:
    df3 = df3[~df3['ngram'].str.contains(i)]
df3.head(n=30)

120781


,ngram,ngram_count,ngram_tokens
2066,environmental flow,735,2
10407,flow regime,340,2
9842,flow alteration,220,2
1977,water scarcity,219,2
9981,ecological response,178,2
9805,natural flow,134,2
9898,water management,94,2
10453,freshwater ecosystem,85,2
2143,water resource,83,2
10439,flow management,81,2
